In [6]:
import pandas as pd
import numpy as np
import nltk
import re

# initializing files

In [7]:
df = pd.read_csv('data.tsv', sep='\t',header=0,on_bad_lines='skip',low_memory=False)
df = df[['review_body','star_rating']]
df = df.dropna()#remove the null values
df = df.reset_index(drop=True)#reset the index

# labeling reviews

In [8]:
df['star_rating'] = df['star_rating'].astype(int)
df = df[df['star_rating'] != 3]
df['label'] = np.where(df['star_rating']>=4,1,0)# 1 = positive, 0 = negative
df['star_rating'].value_counts()

df = df.sample(frac=1).reset_index(drop=True) #shufle

Data = df[df['label'] == 0][:250000]
Data = Data._append(df[df['label'] == 1][:250000])
Data = Data.reset_index(drop=True)
display(Data['label'].value_counts())
Data

label
0    250000
1    250000
Name: count, dtype: int64

,review_body,star_rating,label
0,"Arrived tarnished, returned it without any pro...",1,0
1,There is no such stone as green amethyst. Amet...,1,0
2,I bought this set for my son's wedding. When ...,1,0
3,"very cheap material, it is very synthetic and ...",1,0
4,it wasn't the right one,1,0
...,...,...,...
499995,it was cute and everything. Im 16 and it was a...,4,1
499996,SO BEAUTIFUL!!!,5,1
499997,Beautiful color in just the right size.,5,1
499998,lovely color and quality at a great price!,5,1


# pre-processing

In [9]:
Data['pre_process'] = Data['review_body'].apply(lambda x: " ".join(x.lower() for x in str(x).split())) #convert all the reviews into lower case
from bs4 import BeautifulSoup#remove the html tags and urls from the review
Data['pre_process']=Data['pre_process'].apply(lambda x: BeautifulSoup(x).get_text())
import re
Data['pre_process']=Data['pre_process'].apply(lambda x: re.sub(r"http\S+", "", x))

def contractions(s):#perform the contractions on the reviews
 s = re.sub(r"won't", "will not",s)
 s = re.sub(r"would't", "would not",s)
 s = re.sub(r"could't", "could not",s)
 s = re.sub(r"\'d", " would",s)
 s = re.sub(r"can\'t", "can not",s)
 s = re.sub(r"n\'t", " not", s)
 s= re.sub(r"\'re", " are", s)
 s = re.sub(r"\'s", " is", s)
 s = re.sub(r"\'ll", " will", s)
 s = re.sub(r"\'t", " not", s)
 s = re.sub(r"\'ve", " have", s)
 s = re.sub(r"\'m", " am", s)
 return s
Data['pre_process']=Data['pre_process'].apply(lambda x:contractions(x))
#remove non-alpha characters
Data['pre_process']=Data['pre_process'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))

Data['pre_process']=Data['pre_process'].apply(lambda x: re.sub(' +', ' ', x))#remove extra spaces between the words

from nltk.corpus import stopwords#remove the stopwords
stop = stopwords.words('english')
Data['pre_process']=Data['pre_process'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
Data['pre_process']=Data['pre_process'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))


C:\Users\91639\AppData\Local\Temp\ipykernel_15512\2460694272.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  Data['pre_process']=Data['pre_process'].apply(lambda x: BeautifulSoup(x).get_text())
C:\Users\91639\AppData\Local\Temp\ipykernel_15512\2460694272.py:3: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  Data['pre_process']=Data['pre_process'].apply(lambda x: BeautifulSoup(x).get_text())


In [10]:
Data

,review_body,star_rating,label,pre_process
0,"Arrived tarnished, returned it without any pro...",1,0,arrived tarnished returned without problem bou...
1,There is no such stone as green amethyst. Amet...,1,0,stone green amethyst amethyst definition purpl...
2,I bought this set for my son's wedding. When ...,1,0,bought set son wedding got found cheaply made ...
3,"very cheap material, it is very synthetic and ...",1,0,cheap material synthetic print nice orderd thr...
4,it wasn't the right one,1,0,right one
...,...,...,...,...
499995,it was cute and everything. Im 16 and it was a...,4,1,cute everything im tiny bit big oneday wore go...
499996,SO BEAUTIFUL!!!,5,1,beautiful
499997,Beautiful color in just the right size.,5,1,beautiful color right size
499998,lovely color and quality at a great price!,5,1,lovely color quality great price


# feature extraction  tfidf

In [11]:
from sklearn.model_selection import train_test_split#spliting data into test and train
X_train,X_test,Y_train, Y_test = train_test_split(Data['pre_process'], Data['label'], test_size=0.25, random_state=30)
print("Train: ",X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

print("TFIDF Vectorizer……")#tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)


Train:  (375000,) (375000,) Test:  ((125000,), (125000,))
TFIDF Vectorizer……


In [12]:
print(X_train)

162897                                         nice picture
27737     really liked first received year sitting jewel...
377797                                                 love
28256                                       extremely small
196718    jewelry pouch smaller expected apparently good...
                                ...                        
46220                                         sensative ear
360948    gave christmas present bride wanting pair choc...
48045               cheaply made broke within one use happy
463269                                         good quality
431909    gave best freaind birthday love simple yet pretty
Name: pre_process, Length: 375000, dtype: object


In [13]:
print(Y_train)

162897    0
27737     0
377797    1
28256     0
196718    0
         ..
46220     0
360948    1
48045     0
463269    1
431909    1
Name: label, Length: 375000, dtype: int32


In [14]:
lst = []
lst = vectorizer.idf_

In [15]:
dict = {}
dict = vectorizer.vocabulary_
print(dict)

{'nice': 48864, 'picture': 55016, 'really': 60060, 'liked': 42090, 'first': 28144, 'received': 60298, 'year': 83988, 'sitting': 67634, 'jewelry': 39409, 'box': 9197, 'flower': 28737, 'faded': 26715, 'grey': 32489, 'even': 25333, 'exposed': 26315, 'sunlight': 72152, 'min': 46088, 'ever': 25378, 'sad': 63820, 'love': 43341, 'extremely': 26455, 'small': 68264, 'pouch': 56830, 'smaller': 68293, 'expected': 26006, 'apparently': 3501, 'good': 31629, 'space': 69404, 'perception': 54191, 'also': 2148, 'stiff': 70793, 'high': 34503, 'quality': 59133, 'material': 44871, 'opinion': 51642, 'cross': 17385, 'terrible': 73750, 'cord': 16470, 'black': 8130, 'feel': 27449, 'like': 42079, 'real': 59973, 'leather': 41461, 'gigantic': 30858, 'look': 42852, 'wood': 83000, 'bought': 9109, 'valentine': 79603, 'day': 18580, 'present': 57163, 'wife': 82428, 'friend': 29555, 'commented': 15181, 'beautiful': 6556, 'great': 32275, 'husband': 35706, 'thank': 73889, 'amazon': 2369, 'pendant': 53956, 'photo': 54805,

In [16]:
file1 = open("output.txt","w")
for key, l in zip(dict.keys(), lst):
 st = "{keyn}-  {listn}\n ".format(keyn= key, listn = l)
 file1.write(st)
file1.close()

# training the model

In [17]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)

clf.fit(tf_x_train,Y_train)#fitting the training data into the model
y_test_pred = clf.predict(tf_x_test)#predicting the test data

from sklearn.metrics import classification_report#analyzing the result
report=classification_report(Y_test, y_test_pred,output_dict=True)

In [18]:
report

{'0': {'precision': 0.9158715567248866,
  'recall': 0.9225390338133401,
  'f1-score': 0.9191932045938981,
  'support': 62638},
 '1': {'precision': 0.9216231059994184,
  'recall': 0.9148840640133414,
  'f1-score': 0.9182412205877618,
  'support': 62362},
 'accuracy': 0.91872,
 'macro avg': {'precision': 0.9187473313621526,
  'recall': 0.9187115489133408,
  'f1-score': 0.91871721259083,
  'support': 125000},
 'weighted avg': {'precision': 0.9187409816517534,
  'recall': 0.91872,
  'f1-score': 0.9187182635811728,
  'support': 125000}}

In [19]:
file = open("in.txt", "w")
file.write("very nice")
file.close()

In [20]:

oup = clf.predict()

TypeError: predict() missing 1 required positional argument: 'X'

# Saving the trained model

In [26]:
import pickle

In [27]:
filename = 'trained_model.sav'
filename2 = 'tfidf_v.sav'
pickle.dump(clf, open(filename, 'wb'))
pickle.dump(vectorizer, open(filename2, 'wb'))